In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install wandb
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

# pip install "unsloth @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.1 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [ ]:
# @title prepare data

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
dataset_dict = dataset.train_test_split(test_size=0.004)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 51552
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 208
    })
})

In [ ]:
# @title wandb init
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import os
%env WANDB_WATCH=all
%env WANDB_SILENT=true
os.environ.get("WANDB_SILENT")

env: WANDB_WATCH=all
env: WANDB_SILENT=true


'true'

In [ ]:
import os

from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
import wandb

logging.set_verbosity_info()
project_name = "tiny-llama"
entity = "wandb"
# os.environ["WANDB_LOG_MODEL"] = "checkpoint"

wandb.init(project=project_name, name = "new_run2")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps = 4,
        evaluation_strategy="steps",
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",  # enable logging to W&B
        # run_name="tiny-llama-alpaca-run6",  # name of the W&B run (optional)
        logging_steps=1,  # how often to log to W&B
        logging_strategy = 'steps',
        save_total_limit=2,
    ),
)

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
train/learning_rate,▁
train/loss,▁
eval/loss,2.40896
eval/runtime,7.2294
eval/samples_per_second,1.522


using `logging_steps` to initialize `eval_steps` to 1
PyTorch: setting up devices


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Using auto half precision backend


In [ ]:
trainer.eval_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 11
})

In [ ]:
trainer_stats = trainer.train()
wandb.finish()

***** Running training *****
  Num examples = 3,000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 375
  Number of trainable parameters = 25,231,360
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
1,2.438700,2.408967
2,2.398100,2.408967
3,2.389200,2.408810
4,2.379100,2.408331
5,2.389000,2.407179
6,2.414900,2.405215
7,2.369200,2.402019
8,2.322600,2.397344
9,2.329300,2.390943
10,2.397000,2.382920


***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 

Step,Training Loss,Validation Loss
1,2.438700,2.408967
2,2.398100,2.408967
3,2.389200,2.408810
4,2.379100,2.408331
5,2.389000,2.407179
6,2.414900,2.405215
7,2.369200,2.402019
8,2.322600,2.397344
9,2.329300,2.390943
10,2.397000,2.382920


***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 11
  Batch size = 2
***** Running Evaluation *****
  Num examples = 

Save model

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Inference

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )

inputs = tokenizer(
[
    alpaca_prompt.format(
        "capital of France?", # instruction
        "", # input
        "", # output - leave this blank for a generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Stream Responses

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )

inputs = tokenizer(
[
    alpaca_prompt.format(
        "capital of France?", # instruction
        "", # input
        "", # output - leave this blank for a generation!
    )
]*1, return_tensors = "pt").to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)